In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme(style="ticks")

In [ ]:
load = pd.read_parquet("../data/energy.parquet").asfreq("h")

In [ ]:
sns.lineplot(load)

In [ ]:
sns.lineplot(load.query("date_time.dt.year == 2023"))

## Seasonalities

In [ ]:
# groupby month and show grouped boxplot
load["month"] = load.index.month
load["day"] = load.index.day
load["hour"] = load.index.hour
load["weekday"] = load.index.weekday
sns.boxplot(x="month", y="load", data=load)

In [ ]:
sns.boxplot(x="weekday", y="load", data=load)

In [ ]:
sns.boxplot(x="hour", y="load", data=load)

In [ ]:
load["load"].rolling(52 * 7 * 24).mean().plot()

## Removing the yearly seasonality

In [ ]:
load["logLoad"] = np.log(load["load"])
load["logLoad"].plot()

In [ ]:
from sktime.forecasting.all import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Detrender

model = PolynomialTrendForecaster(degree=2)
detrender = Detrender(forecaster=model)

detrender.fit(load["logLoad"])

In [ ]:
detrender.transform(load["logLoad"]).plot()

In [ ]:
y = load["load"]

In [ ]:
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.darts import DartsXGBModel
from sktime.transformations.series.boxcox import LogTransformer

# from sktime.transformations.series.detrend import Deseasonalizer


quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
# model = DartsLinearRegressionModel(
#     lags=24,
#     output_chunk_length=6,
#     likelihood="quantile",
#     quantiles=quantiles,
#     multi_models=True,
#     kwargs=dict(solver="highs-ipm"),
# )
model = DartsXGBModel(
    lags=24 * 7,
    output_chunk_length=24,
    likelihood="quantile",
    quantiles=quantiles,
    multi_models=False,
    kwargs=dict(n_jobs=1),
)

forecaster = TransformedTargetForecaster(
    [
        LogTransformer(),
        Detrender(PolynomialTrendForecaster(degree=2)),
        model,
    ]
)
forecaster

In [ ]:
import holidays
from sktime.transformations.series.holiday import (
    HolidayFeatures,
)

calender = holidays.country_holidays("DE", subdiv="BW")
holiday_features = HolidayFeatures(
    calender, return_indicator=True, return_dummies=False
)
X = holiday_features.fit_transform(y)

In [ ]:
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.compose import Logger, YtoX

In [ ]:
from probafcst.forecast import get_featurizer

fourier_kwargs = dict(
    sp_list=[24, 24 * 7, 24 * 7 * 52],
    fourier_terms_list=[6, 3, 1],
)

featurizer = get_featurizer(
    fourier_kwargs=fourier_kwargs,
    # datetime_features=["hour_of_day", "hour_of_week", "month_of_year", "is_weekend"],
    datetime_features=["is_weekend"],
    include_holidays=True,
    include_window_summarizer=True,
)

In [ ]:
# from sktime.forecasting.compose import make_reduction
# from sklearn.linear_model import QuantileRegressor
# from sktime.transformations.all import WindowSummarizer, FourierFeatures
# from skpro.regression.multiquantile import MultipleQuantileRegressor

# transformers = [
#     WindowSummarizer(truncate="bfill", lag_feature={"lag": [24, 48], "median": [[24, 24]]}),
#     FourierFeatures(**fourier_kwargs)
# ]
# regressor = MultipleQuantileRegressor(quantile_regressor=QuantileRegressor(solver="highs-ipm"), alpha=quantiles, alpha_name="quantile", n_jobs=-1)

# reducer = make_reduction(estimator=regressor, strategy="direct")
# reducer.fit(y, fh=np.arange(1, 2))

In [ ]:
from sktime.transformations.all import Imputer, Lag


def get_pipeline(forecaster, featurizer, logger_name=None):
    """Get pipeline."""
    steps = [
        ("y_to_x", YtoX()),
        ("featurizer", featurizer),
        ("lags", Lag(lags=[-24, -24 * 7, 0, 24, 24 * 7])),
        ("imputer", Imputer(method="nearest")),
    ]
    if logger_name is not None:
        logger = Logger(logger=logger_name, logger_backend="datalog")
        steps.append(("logger", logger))
    steps.append(("forecaster", forecaster))

    return ForecastingPipeline(steps=steps)


pipe = get_pipeline(forecaster, featurizer, logger_name="test")
pipe.fit(y.loc["2022":])

In [ ]:
from sktime.transformations.compose import DataLog

log = DataLog("test").get_log()
log[-1][1]["X"].tail(25)

In [ ]:
from probafcst.plotting import plot_quantiles

sns.set_theme(style="ticks")
y_pred = pipe.predict_quantiles(np.arange(1, 24 * 7), alpha=quantiles)
plot_quantiles(y.iloc[-24 * 7 :], y_pred)

In [ ]:
%env PYTHONWARNINGS=ignore

In [ ]:
from probafcst.backtest import backtest, get_window_params

wdw = get_window_params(
    n_years_initial_window=2, step_length_days=90, forecast_steps_days=7, freq="h"
)
result = backtest(
    pipe, y, **wdw, quantiles=quantiles, backend="loky", splitter_type="sliding"
)

In [ ]:
result.eval_results

In [ ]:
worst_preds = result.eval_results["test_PinballLoss"].nlargest(5).index
best_preds = result.eval_results["test_PinballLoss"].nsmallest(5).index
worst_preds

In [ ]:
for i, (_, y_test, y_pred_quantiles) in result[2].iloc[worst_preds].iterrows():
    plot_quantiles(y_test, y_pred_quantiles)

In [ ]:
for i, (_, y_test, y_pred_quantiles) in result[2].iloc[best_preds].iterrows():
    plot_quantiles(y_test, y_pred_quantiles)